#Train a MobileNet model on custom dataset from scratch to detect fire 

# Mount dataset from drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! unzip "/content/drive/MyDrive/train.zip"

Streaming output truncated to the last 5000 lines.
  inflating: train/no fire/fire31frame941.jpg  
  inflating: train/no fire/fire31frame942.jpg  
  inflating: train/no fire/fire31frame945.jpg  
  inflating: train/no fire/fire31frame946.jpg  
  inflating: train/no fire/fire31frame95.jpg  
  inflating: train/no fire/fire31frame951.jpg  
  inflating: train/no fire/fire31frame952.jpg  
  inflating: train/no fire/fire31frame954.jpg  
  inflating: train/no fire/fire31frame955.jpg  
  inflating: train/no fire/fire31frame956.jpg  
  inflating: train/no fire/fire31frame957.jpg  
  inflating: train/no fire/fire31frame958.jpg  
  inflating: train/no fire/fire31frame960.jpg  
  inflating: train/no fire/fire31frame966.jpg  
  inflating: train/no fire/fire31frame967.jpg  
  inflating: train/no fire/fire31frame971.jpg  
  inflating: train/no fire/fire31frame972.jpg  
  inflating: train/no fire/fire31frame973.jpg  
  inflating: train/no fire/fire31frame974.jpg  
  inflating: train/no fire/fire31frame

In [3]:
! unzip "/content/drive/MyDrive/test.zip"

Streaming output truncated to the last 5000 lines.
  inflating: test/fire/img_263.jpg   
  inflating: test/fire/img_309.jpg   
  inflating: test/fire/img_336.jpg   
  inflating: test/fire/img_337.jpg   
  inflating: test/fire/img_338.jpg   
  inflating: test/fire/img_347.jpg   
  inflating: test/fire/img_348.jpg   
  inflating: test/fire/img_349.jpg   
  inflating: test/fire/img_350.jpg   
  inflating: test/fire/img_359.jpg   
  inflating: test/fire/img_360.jpg   
  inflating: test/fire/img_361.jpg   
  inflating: test/fire/img_362.jpg   
  inflating: test/fire/img_368.jpg   
  inflating: test/fire/img_369.jpg   
  inflating: test/fire/img_374.jpg   
  inflating: test/fire/img_375.jpg   
  inflating: test/fire/img_379.jpg   
  inflating: test/fire/img_384.jpg   
  inflating: test/fire/img_385.jpg   
  inflating: test/fire/img_388.jpg   
  inflating: test/fire/img_389.jpg   
  inflating: test/fire/img_390.jpg   
  inflating: test/fire/img_399.jpg   
  inflating: test/fire/img_401.jpg   

In [6]:
import os
os.chdir("/content/train")
os.chdir("/content/test")
!pwd

/content/test


## **All necessary Imports**

In [7]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
import tensorflow as tf
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
import h5py
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
import itertools
#for broken data stream error
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
%matplotlib inline

### Image preparation

In [8]:
train_path = '/content/train'
test_path = '/content/test'

In [9]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=1024)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=256, shuffle=False)

Found 16842 images belonging to 2 classes.
Found 7478 images belonging to 2 classes.


### Modify Model

In [10]:
mobile = keras.applications.mobilenet.MobileNet(include_top=False,input_shape=(224,224,3))

17235968/17225924 [==============================] - 0s 0us/step


In [11]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [12]:
#Deleting the last 5 layers and replacing it with a dense softmax layer consisting of 2 nodes: Fire and Non-fire
x = mobile.layers[-4].output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=predictions)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [14]:
#Freezing weights all the layers upto the 4th last layer
#This is done because we will retrain only the last three layers
for layer in model.layers[:-2]:
    layer.trainable = False

In [15]:
model.compile(optimizer=tf.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath ='/content/model/2',save_best_only=True)

In [18]:
model.fit(train_batches, validation_data=test_batches, epochs=5)#, verbose=1, callbacks=[save_best_model]

Epoch 1/5
17/17 [==============================] - 390s 23s/step - loss: 0.2466 - accuracy: 0.9045 - val_loss: 0.2608 - val_accuracy: 0.8914
Epoch 2/5
17/17 [==============================] - 389s 23s/step - loss: 0.2401 - accuracy: 0.9070 - val_loss: 0.2588 - val_accuracy: 0.8898
Epoch 3/5
17/17 [==============================] - 393s 23s/step - loss: 0.2351 - accuracy: 0.9063 - val_loss: 0.2498 - val_accuracy: 0.8936
Epoch 4/5
17/17 [==============================] - 393s 24s/step - loss: 0.2301 - accuracy: 0.9081 - val_loss: 0.2462 - val_accuracy: 0.8926
Epoch 5/5
17/17 [==============================] - 397s 24s/step - loss: 0.2249 - accuracy: 0.9081 - val_loss: 0.2416 - val_accuracy: 0.8968
